In [4]:
#Imports
import redis
import urllib.request
import json
import numpy as np
import pandas as pd
import schedule
import time
import boto3

kinesis = boto3.client('kinesis', region_name='us-east-1')


#Connect to Redis-DataStore
REDIS = redis.Redis(host='data_store')

In [2]:
%env AWS_SECRET_ACCESS_KEY

'82pgo1H+eurVJr548ZNAuYbqzCewoSojQ4B90TWJ'

In [5]:
kinesis.list_streams()

{'HasMoreStreams': False,
 'ResponseMetadata': {'HTTPHeaders': {'content-length': '51',
   'content-type': 'application/x-amz-json-1.1',
   'date': 'Wed, 31 Jan 2018 19:42:11 GMT',
   'server': 'Apache-Coyote/1.1',
   'x-amz-id-2': 'CVOdVlx2sHKVk7joEW3ju/Bjq9Ua5p6BALiphZm7U1KHI+dKGYp7icwmohHY/JLzuwdEU6bY4ibiT58DEbg7wBVLZWqxmfwlP3PUsrZROrE=',
   'x-amzn-requestid': 'f86afce4-5ce6-a3f6-aa86-e2b0c626b762'},
  'HTTPStatusCode': 200,
  'RequestId': 'f86afce4-5ce6-a3f6-aa86-e2b0c626b762',
  'RetryAttempts': 0},
 'StreamNames': ['BotoDemo']}

### Redis Test

In [4]:
REDIS.get('my_incrementer')

b'42'

In [5]:
#To pass a dictionary:
import numpy as np
import json

model_params = {
    'C': list(np.logspace(-3,3,7)),
    'penalty': 'l1',
    'solver' : 'newton-cg'
}

REDIS.set('model_params', json.dumps(model_params))

True

#### Build Company dataframe

In [20]:
companies = json.loads(REDIS.get('companies').decode())
stock_tickers = list(companies.keys())
stock_tickers

['AAPL', 'FB', 'GOOG']

In [ ]:
#Manager Log
# Need to send Number of Companies, Rate

### IEX Code

In [12]:
#Want to create a simple list, and then convert the time stamps
attributes = ['latestUpdate',
             'companyName',
             'latestPrice',
              'latestVolume',
              'marketCap',
              'open',
              'previousClose',
              'sector',
              'high',
              'low',
              'ytdChange',
              'peRatio',
              'week52High',
              'week52Low'
               ]
def filter_stock_attributes(data):
    output = {}
    #Choose filter attributes
    for element in attributes:
        output[element]=data[element]
    
    #Convert time
    ctime = output['latestUpdate']/1000
    new_time = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(ctime))
    output['latestUpdate']= new_time
    
    return output

#Create function to fetch the stock data. This is to prepare for the schedule.
def fetch_stock_data(stocks=[]):
    
    for ticker in stocks:
        url = "https://api.iextrading.com/1.0/stock/{}/quote".format(ticker)
        response = urllib.request.urlopen(url)
        str_response = response.read().decode('utf-8')
        obj = json.loads(str_response)
        filtered = filter_stock_attributes(obj)
        #Add counter to count stocks. 
        REDIS.incr('IEX_Stock_Count')
        #<----- Insert to Kinesis Stream ------->
        response = kinesis.put_record(StreamName="BotoDemo", Data=json.dumps(filtered), PartitionKey="partitionkey")
        print('---------------------------------')
        print(response)
        print(filtered)

In [13]:
#Read Data-Store
companies = json.loads(REDIS.get('companies').decode())
stock_tickers = list(companies.keys())
REDIS.set('IEX_Stock_Count', 0)

#Setup Schedule
schedule.clear()
schedule.every(10).seconds.do(fetch_stock_data, stocks=stock_tickers)

#Execute
while True:
    schedule.run_pending()
    time.sleep(1)

---------------------------------
{'ShardId': 'shardId-000000000000', 'SequenceNumber': '49581293592208739795471062951283188790049960114553618434', 'ResponseMetadata': {'RequestId': 'd55efdd0-dc17-e2ae-87b2-dde807393cfc', 'HTTPStatusCode': 200, 'HTTPHeaders': {'server': 'Apache-Coyote/1.1', 'x-amzn-requestid': 'd55efdd0-dc17-e2ae-87b2-dde807393cfc', 'x-amz-id-2': 'bxKH1BAvZaJnp+HEc8zGmhJQ+2SqzoWI7HIBhtL0uQ3IavscH4335UPOf8q1XGFEBilj2pADoWwBpm2NmJMl9K9aBy19JVUv6QeOLzbF9Qc=', 'content-type': 'application/x-amz-json-1.1', 'content-length': '110', 'date': 'Wed, 31 Jan 2018 19:50:16 GMT'}, 'RetryAttempts': 0}}
{'latestUpdate': '2018-01-31 19:50:09', 'companyName': 'Apple Inc.', 'latestPrice': 166.86, 'latestVolume': 21689381, 'marketCap': 848826164160, 'open': 166.95, 'previousClose': 166.97, 'sector': 'Technology', 'high': 168.442, 'low': 166.5, 'ytdChange': -0.03070939277835825, 'peRatio': 18.14, 'week52High': 180.1, 'week52Low': 120.62}
---------------------------------
{'ShardId': 'shard

---------------------------------
{'ShardId': 'shardId-000000000000', 'SequenceNumber': '49581293592208739795471062951294069122426493976149229570', 'ResponseMetadata': {'RequestId': 'fc9d4ae1-716b-d112-ae71-6ab91557c84a', 'HTTPStatusCode': 200, 'HTTPHeaders': {'server': 'Apache-Coyote/1.1', 'x-amzn-requestid': 'fc9d4ae1-716b-d112-ae71-6ab91557c84a', 'x-amz-id-2': 'Eri4PE98UKmsg626HGnG8ZWvMpWA3tAz04Cct/gVPAzJ2fXCgTIilsPf/XOJSp1kHky+WqGOuPmdKwAvJlW6daQu5jfBw94S0uFkXVS/UCg=', 'content-type': 'application/x-amz-json-1.1', 'content-length': '110', 'date': 'Wed, 31 Jan 2018 19:50:48 GMT'}, 'RetryAttempts': 0}}
{'latestUpdate': '2018-01-31 19:50:39', 'companyName': 'Apple Inc.', 'latestPrice': 166.79, 'latestVolume': 21740984, 'marketCap': 848470070240, 'open': 166.95, 'previousClose': 166.97, 'sector': 'Technology', 'high': 168.442, 'low': 166.5, 'ytdChange': -0.03070939277835825, 'peRatio': 18.13, 'week52High': 180.1, 'week52Low': 120.62}
---------------------------------
{'ShardId': 'shard

---------------------------------
{'ShardId': 'shardId-000000000000', 'SequenceNumber': '49581293592208739795471062951304949454803027906464317442', 'ResponseMetadata': {'RequestId': 'e8907b31-856c-b48f-ba7c-5b48e5e1e10a', 'HTTPStatusCode': 200, 'HTTPHeaders': {'server': 'Apache-Coyote/1.1', 'x-amzn-requestid': 'e8907b31-856c-b48f-ba7c-5b48e5e1e10a', 'x-amz-id-2': 'mgv6vXI5JxV1y3Nr4t643iwlfT3z3DOcOk72PeW6DM37cnn9zMfOeyYWPMzGgT3JDfuZg1FBUcG8Dp23xKXnY+Ggjdc5gWTZ', 'content-type': 'application/x-amz-json-1.1', 'content-length': '110', 'date': 'Wed, 31 Jan 2018 19:51:21 GMT'}, 'RetryAttempts': 0}}
{'latestUpdate': '2018-01-31 19:51:13', 'companyName': 'Apple Inc.', 'latestPrice': 166.84, 'latestVolume': 21784343, 'marketCap': 848724423040, 'open': 166.95, 'previousClose': 166.97, 'sector': 'Technology', 'high': 168.442, 'low': 166.5, 'ytdChange': -0.03070939277835825, 'peRatio': 18.13, 'week52High': 180.1, 'week52Low': 120.62}
---------------------------------
{'ShardId': 'shardId-000000000

---------------------------------
{'ShardId': 'shardId-000000000000', 'SequenceNumber': '49581293592208739795471062951315829787179561836779405314', 'ResponseMetadata': {'RequestId': 'e9597078-d6d9-ce43-bbb5-50e2e5238265', 'HTTPStatusCode': 200, 'HTTPHeaders': {'server': 'Apache-Coyote/1.1', 'x-amzn-requestid': 'e9597078-d6d9-ce43-bbb5-50e2e5238265', 'x-amz-id-2': 'KZuG2mnVX7JfRK6RhfPWGCFSeJTr4dxRc1uZ5/t+8FgzQO13SM5khyFgz1FhucdfcmE6erdV7+Tpsx3k5YFi/UGXIUH/7RQC', 'content-type': 'application/x-amz-json-1.1', 'content-length': '110', 'date': 'Wed, 31 Jan 2018 19:51:53 GMT'}, 'RetryAttempts': 0}}
{'latestUpdate': '2018-01-31 19:51:50', 'companyName': 'Apple Inc.', 'latestPrice': 166.91, 'latestVolume': 21828651, 'marketCap': 849080516960, 'open': 166.95, 'previousClose': 166.97, 'sector': 'Technology', 'high': 168.442, 'low': 166.5, 'ytdChange': -0.03070939277835825, 'peRatio': 18.14, 'week52High': 180.1, 'week52Low': 120.62}
---------------------------------
{'ShardId': 'shardId-000000000

---------------------------------
{'ShardId': 'shardId-000000000000', 'SequenceNumber': '49581293592208739795471062951326710119556095698375016450', 'ResponseMetadata': {'RequestId': 'd39c48f7-2f7a-ad27-8170-684d4b9f1264', 'HTTPStatusCode': 200, 'HTTPHeaders': {'server': 'Apache-Coyote/1.1', 'x-amzn-requestid': 'd39c48f7-2f7a-ad27-8170-684d4b9f1264', 'x-amz-id-2': 'LGHwS1gAJQ8ftmUdIRvD1ZJ2w8A0jFvb3WPM8MbwJ/6hUNIO82WOQDSpNG8RIzgWmK9uLKag7slO8UvqP++pmhPNr8Q93kj6', 'content-type': 'application/x-amz-json-1.1', 'content-length': '110', 'date': 'Wed, 31 Jan 2018 19:52:26 GMT'}, 'RetryAttempts': 0}}
{'latestUpdate': '2018-01-31 19:52:19', 'companyName': 'Apple Inc.', 'latestPrice': 166.96, 'latestVolume': 21856616, 'marketCap': 849334869760, 'open': 166.95, 'previousClose': 166.97, 'sector': 'Technology', 'high': 168.442, 'low': 166.5, 'ytdChange': -0.03070939277835825, 'peRatio': 18.15, 'week52High': 180.1, 'week52Low': 120.62}
---------------------------------
{'ShardId': 'shardId-000000000

KeyboardInterrupt: 